In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xlrd
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY
file_path = 'data.xlsx'
df = pd.read_excel(file_path)
wb = xlrd.open_workbook(file_path) 
d = np.zeros((30,30))
p = np.zeros((30,30))
xc = np.zeros((4))
sheet_d = wb.sheet_by_index(0) 
sheet_p = wb.sheet_by_index(1)
for i in range(30):
    for j in range(30):
        d[i][j] = sheet_d.cell_value(i, j)
        p[i][j] = sheet_p.cell_value(i, j)
import sys
c = d

In [2]:
"""Vehicles Routing Problem (VRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(data,x):
    """Stores the data for the problem."""
    
    data['distance_matrix'] = d
    data['num_vehicles'] = x
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for volunteer {}:\n'.format(vehicle_id+1)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index)+1)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index)+1)
        plan_output += 'Distance of the route: {}km\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}km'.format(max_route_distance))



def main():
    
    for i in range(1,30):
        
        data = {}
        """Solve the CVRP problem."""
        # Instantiate the data problem.
        data = create_data_model(data,i)

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)


        # Create and register a transit callback.
        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add Distance constraint.
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            400,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)

        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # Setting first solution metaheuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
        search_parameters.time_limit.seconds = 5

        solution = routing.SolveWithParameters(search_parameters)
        # Print solution on console.
        if solution == None:
            print("Fail! Number of Valunteers : " , i, " ")


            continue
        else:
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
            search_parameters.time_limit.seconds = 10

            solution = routing.SolveWithParameters(search_parameters)
            print("Sucsess! Number of Volunteers : " , i, " ")
            print_solution(data, manager, routing, solution)    
            break;


if __name__ == '__main__':
    main()

Fail! Number of Valunteers :  1  
Fail! Number of Valunteers :  2  
Sucsess! Number of Volunteers :  3  
Route for volunteer 1:
 1 ->  10 ->  24 ->  29 ->  5 ->  20 ->  23 ->  4 ->  3 ->  30 -> 1
Distance of the route: 383km

Route for volunteer 2:
 1 ->  2 ->  16 ->  18 ->  22 ->  17 ->  14 ->  6 ->  27 ->  19 ->  11 ->  21 -> 1
Distance of the route: 382km

Route for volunteer 3:
 1 ->  9 ->  12 ->  13 ->  26 ->  28 ->  25 ->  8 ->  7 ->  15 -> 1
Distance of the route: 393km

Maximum of the route distances: 393km


Sec:  100  
Route for volunteer 1:
 1 ->  11 ->  19 ->  26 ->  13 ->  12 ->  28 ->  25 ->  8 ->  7 ->  15 -> 1
Distance of the route: 387km

Route for volunteer 2:
 1 ->  2 ->  16 ->  18 ->  22 ->  17 ->  14 ->  6 ->  27 ->  9 ->  21 -> 1
Distance of the route: 364km

Route for volunteer 3:
 1 ->  30 ->  3 ->  4 ->  23 ->  20 ->  5 ->  29 ->  24 ->  10 -> 1
Distance of the route: 383km

Maximum of the route distances: 387km
Sec:  200  
Route for volunteer 1:
 1 ->  2 ->  16 ->  22 ->  18 ->  13 ->  12 ->  26 ->  19 ->  11 ->  21 -> 1
Distance of the route: 382km

Route for volunteer 2:
 1 ->  15 ->  7 ->  8 ->  25 ->  28 ->  17 ->  14 ->  6 ->  27 ->  9 -> 1
Distance of the route: 379km

Route for volunteer 3:
 1 ->  30 ->  3 ->  4 ->  23 ->  20 ->  5 ->  29 ->  24 ->  10 -> 1
Distance of the route: 383km

Maximum of the route distances: 383km
Sec:  300  
Route for volunteer 1:
 1 ->  2 ->  16 ->  22 ->  18 ->  13 ->  12 ->  26 ->  19 ->  11 ->  21 -> 1
Distance of the route: 382km

Route for volunteer 2:
 1 ->  15 ->  7 ->  8 ->  25 ->  28 ->  17 ->  14 ->  6 ->  27 ->  9 -> 1
Distance of the route: 379km

Route for volunteer 3:
 1 ->  30 ->  3 ->  4 ->  23 ->  20 ->  5 ->  29 ->  24 ->  10 -> 1
Distance of the route: 383km

Maximum of the route distances: 383km
Sec:  400  
Route for volunteer 1:
 1 ->  2 ->  16 ->  22 ->  18 ->  13 ->  12 ->  26 ->  19 ->  11 ->  21 -> 1
Distance of the route: 382km

Route for volunteer 2:
 1 ->  15 ->  7 ->  8 ->  25 ->  28 ->  17 ->  14 ->  6 ->  27 ->  9 -> 1
Distance of the route: 379km

Route for volunteer 3:
 1 ->  30 ->  3 ->  4 ->  23 ->  20 ->  5 ->  29 ->  24 ->  10 -> 1
Distance of the route: 383km

Maximum of the route distances: 383km
Sec:  500  
Route for volunteer 1:
 1 ->  2 ->  16 ->  22 ->  18 ->  13 ->  12 ->  26 ->  19 ->  11 ->  21 -> 1
Distance of the route: 382km

Route for volunteer 2:
 1 ->  15 ->  7 ->  8 ->  25 ->  28 ->  17 ->  14 ->  6 ->  27 ->  9 -> 1
Distance of the route: 379km

Route for volunteer 3:
 1 ->  30 ->  3 ->  4 ->  23 ->  20 ->  5 ->  29 ->  24 ->  10 -> 1
Distance of the route: 383km

Maximum of the route distances: 383km
Sec:  600  
Route for volunteer 1:
 1 ->  2 ->  16 ->  22 ->  18 ->  13 ->  12 ->  26 ->  19 ->  11 ->  21 -> 1
Distance of the route: 382km

Route for volunteer 2:
 1 ->  15 ->  7 ->  8 ->  25 ->  28 ->  17 ->  14 ->  6 ->  27 ->  9 -> 1
Distance of the route: 379km

Route for volunteer 3:
 1 ->  30 ->  3 ->  4 ->  23 ->  20 ->  5 ->  29 ->  24 ->  10 -> 1
Distance of the route: 383km

Maximum of the route distances: 383km